# hw 3
## due **Feb. 18**, 12:30pm

Feb 16 is a wellness day.

In this assignment we'll explore some logical operations if/then/else and will explore summarizing survey data using subgroups defined by variables --- in other words, subgroups defined by people's responses to certain questions in the survey.

### data
This assignment uses the `shed_survey` dataset.

Match the names of the variables with those provided in the official codebook for this dataset.

Codebook: https://www.federalreserve.gov/consumerscommunities/files/SHED-2019-codebook.pdf

2019 SHED survey dataset https://www.federalreserve.gov/consumerscommunities/shed_data.htm https://www.federalreserve.gov/publications/2020-update-economic-well-being-of-us-households-preface.htm



### Q1

After loading the dataset, 

- **create** a new column called `dummy` which as value 1 if EF3_c = "yes" and 0 otherwise. *pandas people* can use True instead of 1 and False instead of 0, which will behave the same for this question.
- **count** the number of missing values (excel: blank, pandas: NA) in EF3_c
- **count** the number of missing values in `dummy`
- **write** one quick phrase explaining why those numbers differ

The blanks represent cases in which the respondent refused to answer yes or no.

This is a useless variable just to make a point: Your actions can sometimes have unintended consequences for understanding missing data!

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
d = pd.read_csv('https://github.com/brendanrbrown/stor155_sp21/raw/main/data/shed_survey.csv')

In [3]:
d = d.assign(dummy = d.EF3_c.eq('yes'))

In [4]:
d.loc[:, ['EF3_c', 'dummy']].isna().sum()

EF3_c    57
dummy     0
dtype: int64

In [5]:
# aside: if you wanted to recode for some reason while preserving missing you could do
# d.EF3_c.str.match('yes')
# or equivalently
# d.loc[:, 'EF3_c'].str.match('yes')

# shows that it preserves NA
d.EF3_c.str.match('yes').isna().sum()

57

### Q2

After loading the dataset, 

- **create** a new column called `n_yes` which counts the number of 'yes' answers in the EF3 series of variables for each row
- **create** a new column called `any_cash_equiv` which equals 1 (pandas: you can use True here too) if either EF3_a = 'yes' or EF3_c = 'yes' or both
- **create** a new column called `only_cash_equiv` which equals 1 (pandas: you can use True here too) if either EF3_a = 'yes' or EF3_c = 'yes' (or both) are true but none of the other EF3 variables is 'yes'

Breaking this down a bit:

`any_cash_equiv` is a variable that counts whether the respondent said they would pay with cash (EF3_c = 'yes') or equivalent (EF3_a = 'yes') regardless of what other payment methods they chose in addition.

`only_cash_equiv` counts whether the respondent said they'd pay with cash or equivalent but does *not* count observations for which some other payment method also was listed.

In [6]:
# a few new ideas here if you have never programmed before

# d.columns.str.match('EF3') returns True if the word EF3 appears in the column name
# which can be passed to .loc[] to access those columns

# as mentioned in class you are allowed to cheat in creating the only_cash_equiv
# point is to show you can use previously created variables within assign using 'lambda' anonymous function

# summing over True/False will treat them like 1/0 respectively

# the apply statement was explained in class. check out the docs and come talk to me.

d = d.assign(n_yes = d.loc[:, d.columns.str.match('EF3')].apply(lambda x: x.eq('yes').sum(), axis = 1),
             any_cash_equiv = d.loc[:, ['EF3_a', 'EF3_c']].apply(lambda x: x.eq('yes').any(), axis = 1), 
             # this is cheating. see real answer below
             # x now refers to the data frame after the changes above within this assign statement
             only_cash_equiv = lambda x: x.n_yes.le(2) & x.any_cash_equiv)

In [7]:
d.head()

,CaseID,Duration,B2,B7_a,B7_b,D1A,D1E,EF3_a,EF3_b,EF3_c,EF3_d,EF3_e,EF3_f,EF3_g,EF3_h,dummy,n_yes,any_cash_equiv,only_cash_equiv
0,1,24665,doing okay,poor,poor,no,yes,yes,no,yes,no,no,no,yes,no,True,3,True,False
1,2,721,doing okay,good,good,no,yes,no,no,yes,no,no,no,no,no,True,1,True,True
2,3,937,doing okay,good,good,yes,no,yes,no,no,no,no,no,no,no,False,1,True,True
3,4,1066,just getting by,only fair,good,yes,no,no,no,yes,no,no,no,no,no,True,1,True,True
4,5,1441,living comfortably,good,good,no,no,no,no,yes,no,no,no,no,no,True,1,True,True


### Alternatives

In [8]:
# I demonstrate apply there intentionally, but
# in fact you can do the first assignments in a more compact way
# using the fact that .eq() also is a data frame method and sum() and any() can apply across axes

d = d.assign(n_yes = d.loc[:, d.columns.str.match('EF3')].eq('yes').sum(axis = 1),
             any_cash_equiv = d.loc[:, ['EF3_a', 'EF3_c']].eq('yes').any(axis = 1),
             only_cash_equiv = lambda x: x.n_yes.le(2) & x.any_cash_equiv)

In [9]:
# above is a fine answer
# this is how you actually would create only_cash_equiv variable
# careful! any() here is now a data frame method, not the series method used above
# so you need to ask which axis it is applying itself to

only_cash = d.any_cash_equiv & (~d.loc[:, d.columns.str.match('EF3_[bdefgh]')].eq('yes').any(axis = 1))

In [10]:
# there's a difference!
only_cash.sum(), d.only_cash_equiv.sum()

(8060, 8532)

For those of you interested in computational efficiency, note the more wordy `.loc[]` assignment is in general a bit better

In [11]:
d = pd.read_csv('https://github.com/brendanrbrown/stor155_sp21/raw/main/data/shed_survey.csv')

In [12]:
%timeit -r 15 d.assign(n_yes = d.loc[:, d.columns.str.match('EF3')].eq('yes').sum(axis = 1), any_cash_equiv = d.loc[:, ['EF3_a', 'EF3_c']].eq('yes').sum(axis = 1), only_cash_equiv = lambda x: x.n_yes.le(2) & x.any_cash_equiv)

7.93 ms ± 445 µs per loop (mean ± std. dev. of 15 runs, 100 loops each)


In [13]:
d = pd.read_csv('https://github.com/brendanrbrown/stor155_sp21/raw/main/data/shed_survey.csv')

In [14]:
%timeit -r 15 d.loc[:, 'n_yes'] = d.loc[:, d.columns.str.match('EF3')].eq('yes').sum(axis = 1); d.loc[:, 'any_cash_equiv'] = d.loc[:, ['EF3_a', 'EF3_c']].eq('yes').sum(axis = 1); d.loc[:, 'only_cash_equiv'] = d.loc[:, 'any_cash_equiv'] & d.loc[:, 'n_yes'].le(2)

6.9 ms ± 35.2 µs per loop (mean ± std. dev. of 15 runs, 100 loops each)


### Q3

- **Give the within-group averages** for the three variables you created in Q2, for groups given by the values in variable B2.

For example, the average `n_yes` for the group of rows for which B2 = `doing ok` is the sum of `n_yes` values for observations in that group divided by the number of observations in that group.

Excel users: You will need to use a pivot table.

Pandas users: You will need to use the .groupby() method followed by the .agg() method.


In [15]:
d.loc[:, ['B2', 'n_yes', 'any_cash_equiv', 'only_cash_equiv']].groupby('B2').agg('mean')

,n_yes,any_cash_equiv,only_cash_equiv
B2,,,
doing okay,1.279626,0.886902,0.659459
finding it difficult to get by,1.450980,0.177778,0.116340
just getting by,1.413520,0.443357,0.319347
living comfortably,1.182984,1.090254,0.808913


### Q4

- **Give the within-group average** for the three variables you created in Q2, but this time for groups given by the values in variables `D1A` and `D1E`.
- Also **count** the number of observations within each of those groups.

A group defined by two variables will be given by each unique pair of values from those variables. For example, D1A = 'yes' and D1E = 'yes' will be one group and D1A = 'yes' and D1E = 'no' will be another.

Excel users: Create a *separate pivot table* for this answer, to preserve your answers to the previous question.

In [16]:
d.loc[:, ['D1A', 'D1E', 'n_yes', 'any_cash_equiv', 'only_cash_equiv']].groupby(['D1A', 'D1E']).agg(['mean', 'count'])

n_yes       any_cash_equiv       only_cash_equiv      
             mean count           mean count            mean count
D1A D1E                                                           
no  no   1.200455  3517       0.900768  3517        0.685812  3517
    yes  1.317323  1270       0.511024  1270        0.384252  1270
yes no   1.237500  4960       0.958065  4960        0.708468  4960
    yes  1.456549  2405       0.674428  2405        0.466944  2405